# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe09405ac40>
├── 'a' --> tensor([[ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465]])
└── 'x' --> <FastTreeValue 0x7fe09405a8e0>
    └── 'c' --> tensor([[ 1.1390, -2.2182,  0.1576,  0.1473],
                        [-0.4362,  0.9242,  0.4162,  0.0590],
                        [ 0.6509,  0.4018, -0.5972,  0.9638]])

In [4]:
t.a

tensor([[ 0.1270, -1.2209, -0.3810],
        [-0.0637,  2.0873, -0.3465]])

In [5]:
%timeit t.a

70.6 ns ± 0.0877 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe09405ac40>
├── 'a' --> tensor([[ 1.4286, -1.9683,  0.7937],
│                   [-1.7136,  1.3492, -1.8783]])
└── 'x' --> <FastTreeValue 0x7fe09405a8e0>
    └── 'c' --> tensor([[ 1.1390, -2.2182,  0.1576,  0.1473],
                        [-0.4362,  0.9242,  0.4162,  0.0590],
                        [ 0.6509,  0.4018, -0.5972,  0.9638]])

In [7]:
%timeit t.a = new_value

71.5 ns ± 0.0883 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465]]),
    x: Batch(
           c: tensor([[ 1.1390, -2.2182,  0.1576,  0.1473],
                      [-0.4362,  0.9242,  0.4162,  0.0590],
                      [ 0.6509,  0.4018, -0.5972,  0.9638]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1270, -1.2209, -0.3810],
        [-0.0637,  2.0873, -0.3465]])

In [11]:
%timeit b.a

65 ns ± 0.0589 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0811, -0.8068,  2.0994],
               [ 0.7485, -2.2535, -1.1902]]),
    x: Batch(
           c: tensor([[ 1.1390, -2.2182,  0.1576,  0.1473],
                      [-0.4362,  0.9242,  0.4162,  0.0590],
                      [ 0.6509,  0.4018, -0.5972,  0.9638]]),
       ),
)

In [13]:
%timeit b.a = new_value

541 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

878 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 24.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 410 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 699 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fdfc92cb8b0>
├── 'a' --> tensor([[[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]],
│           
│                   [[ 0.1270, -1.2209, -0.3810],
│                    [-0.0637,  2.0873, -0.3465]]])
└── 'x' --> <FastTreeValue 0x7fdfc92c9f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34.1 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe09913ae80>
├── 'a' --> tensor([[ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465],
│                   [ 0.1270, -1.2209, -0.3810],
│                   [-0.0637,  2.0873, -0.3465]])
└── 'x' --> <FastTreeValue 0x7fe0990d95b0>
    └── 'c' --> tensor([[ 1.1390, -2.2182,  0.1576,  0.1473],
                        [-0.4362,  0.9242,  0.4162,  0.0590],
                 

In [23]:
%timeit t_cat(trees)

32.1 µs ± 36.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.3 µs ± 27.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]],
       
               [[ 0.1270, -1.2209, -0.3810],
                [-0.0637,  2.0873, -0.3465]]]),
    x: Batch(
           c: tensor([[[ 1.1390, -2.2182,  0.1576,  0.1473],
                       [-0.4362,  0.9242,  0.4162,  0.0590],
                       [ 0.6509,  0.4018, -0.5972,  0.9638]],
         

In [26]:
%timeit Batch.stack(batches)

83.8 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465],
               [ 0.1270, -1.2209, -0.3810],
               [-0.0637,  2.0873, -0.3465]]),
    x: Batch(
           c: tensor([[ 1.1390, -2.2182,  0.1576,  0.1473],
                      [-0.4362,  0.9242,  0.4162,  0.0590],
                      [ 0.6509,  0.4018, -0.5972,  0.9638],
                      [ 1.1390, -2.2182,  0.1576,  0.1473],
                      [-0.4362,  

In [28]:
%timeit Batch.cat(batches)

157 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

351 µs ± 5.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
